In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd  # Library for applying functions to dataframes in parallel 
from dask.diagnostics import ProgressBar  # To check progress of dask computation(s)

In [3]:
# Original file too large to upload to github. Dataset processed locally.
# Dataset location: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents

data = pd.read_csv('US_Accidents_March23.csv')

In [4]:
class_distribution = data['Severity'].value_counts()

# To get a percentage distribution, you can use:
class_distribution_percent = data['Severity'].value_counts(normalize=True) * 100

# Printing the results
print("Class Distribution Count:\n", class_distribution)
print("\nClass Distribution Percentage:\n", class_distribution_percent)

Class Distribution Count:
 2    6156981
3    1299337
4     204710
1      67366
Name: Severity, dtype: int64

Class Distribution Percentage:
 2    79.667017
3    16.812510
4     2.648804
1     0.871669
Name: Severity, dtype: float64


In [5]:
data.columns

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [6]:
# Want to remove datapoints with missing values in the following columns:
columns_to_check = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Sunrise_Sunset']

# Drop rows with missing values in the specified columns
data = data.dropna(subset=columns_to_check)

data.shape

(7487604, 46)

# Want to remove data points where date/time value is incorrect
Initially, tried the approach below but was taking much too long

# Function to check if a date string is in the correct format
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str)
        return True
    except ValueError:
        return False

# Filter out rows with invalid 'Start_Time' or 'End_Time'
valid_start_time = data['Start_Time'].apply(is_valid_date)
valid_end_time = data['End_Time'].apply(is_valid_date)
data_clean = data[valid_start_time & valid_end_time]

Will instead use the Dask library to perform this task in parallel 

Tried this approach as well. Taking much too long.

dask_data = dd.from_pandas(data, npartitions=4)

# Filter out rows with invalid 'Start_Time' or 'End_Time'
valid_start_time = dask_data['Start_Time'].map_partitions(lambda x: x.apply(is_valid_date), meta=bool)
valid_end_time = dask_data['End_Time'].map_partitions(lambda x: x.apply(is_valid_date), meta=bool)

dask_data_clean = dask_data[valid_start_time & valid_end_time]

# Dask performs computations in 'lazy' manner, must call compute() to trigger actual computation
# The compute function returns a pandas dataframe

# Wrapping computation in a ProgressBar context manager
with ProgressBar():
    dask_data_clean = dask_data_clean.compute()

In [7]:
# Function to check if a date string is in the correct format
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str)
        return True
    except ValueError:
        return False

In [8]:
# Now we will try filtering at the point of subset creation. This should be much more efficient than iterating over the entire original dataset

def sample_and_filter(df, class_value, sample_size, date_columns):
    # Sample the DataFrame
    sample_df = df[df['Severity'] == class_value].sample(sample_size, random_state=42)
    
    # Filter out invalid dates
    for col in date_columns:
        sample_df = sample_df[sample_df[col].apply(is_valid_date)]
    
    return sample_df

# Columns to check for valid dates
date_columns = ['Start_Time', 'End_Time']

# Sample size for each class
sample_size = 20000

# Creating the samples
class_1_sample = sample_and_filter(data, 1, sample_size, date_columns)
class_2_sample = sample_and_filter(data, 2, sample_size, date_columns)
class_3_sample = sample_and_filter(data, 3, sample_size, date_columns)
class_4_sample = sample_and_filter(data, 4, sample_size, date_columns)

# Combine the samples into a balanced dataset
balanced_data = pd.concat([class_1_sample, class_2_sample, class_3_sample, class_4_sample])

# Shuffle the dataset (optional but recommended)
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)


In [9]:
# Testing our approach 
class_distribution = balanced_data['Severity'].value_counts()

# To get a percentage distribution, you can use:
class_distribution_percent = balanced_data['Severity'].value_counts(normalize=True) * 100

# Printing the results
print("Class Distribution Count:\n", class_distribution)
print("\nClass Distribution Percentage:\n", class_distribution_percent)

Class Distribution Count:
 3    20000
4    20000
1    20000
2    20000
Name: Severity, dtype: int64

Class Distribution Percentage:
 3    25.0
4    25.0
1    25.0
2    25.0
Name: Severity, dtype: float64


In [11]:
# Save the balanced dataset to a CSV file
balanced_data.to_csv('US_Accidents_Balanced_Sample.csv', index=False)